## Import useful libraries

In [44]:
from bs4 import BeautifulSoup

## Read html an parse cells line by line

In [45]:
with open ('my-iso-code.txt','r') as fp: #'r' Lecture
    html = fp.read()
    soup = BeautifulSoup(html,'html5lib') #Récuper les données plus facilement

In [46]:
#len(soup.find_all('table'))
#'tr' == ligne
#'td' == cellule

In [48]:
trs = soup.find_all('tr') #'trs' comprend l'ensemble des lignes du tableau de l'html 'tr'
print(len(trs))
for tr in trs:
    values = [td.string for td in tr.find_all('td')]
    print(values)

249
['Afghanistan', "Afghanistan (l')", 'AF', 'AFG', '004']
['Ã…land Islands', 'Ã…land(les ÃŽles)', 'AX', 'ALA', '248']
['Albania', "Albanie (l')", 'AL', 'ALB', '008']
['Algeria', "AlgÃ©rie (l')", 'DZ', 'DZA', '012']
['American Samoa', 'Samoa amÃ©ricaines (les)', 'AS', 'ASM', '016']
['Andorra', "Andorre (l')", 'AD', 'AND', '020']
['Angola', "Angola (l')", 'AO', 'AGO', '024']
['Anguilla', 'Anguilla', 'AI', 'AIA', '660']
['Antarctica', "Antarctique (l')", 'AQ', 'ATA', '010']
['Antigua and Barbuda', 'Antigua-et-Barbuda', 'AG', 'ATG', '028']
['Argentina', "Argentine (l')", 'AR', 'ARG', '032']
['Armenia', "ArmÃ©nie (l')", 'AM', 'ARM', '051']
['Aruba', 'Aruba', 'AW', 'ABW', '533']
['Australia', "Australie (l')", 'AU', 'AUS', '036']
['Austria', "Autriche (l')", 'AT', 'AUT', '040']
['Azerbaijan', "AzerbaÃ¯djan (l')", 'AZ', 'AZE', '031']
['Bahamas (the)', 'Bahamas (les)', 'BS', 'BHS', '044']
['Bahrain', 'BahreÃ¯n', 'BH', 'BHR', '048']
['Bangladesh', 'Bangladesh (le)', 'BD', 'BGD', '050']
['Barb

In [50]:
#for tr in trs:
#    for td in tr.find_all('td'):
#        print(td)

In [51]:
countries = []
iso_code = []
for tr in trs:
    values = [td.string for td in tr.find_all('td')]
    countries.append((str(values[0])))
    iso_code.append((str(values[3])))
countries = countries[1:-1] #ne prend pas la première est dernière valeur
#print(countries)

In [52]:
import pickle
with open('countries','wb+') as fp:
    pickle.dump(countries, fp)

In [ ]:
with open ('countries','rb') as fp:
    countries = pickle.load(fp)
    
results = []
header = ['name','code']
header += indicator_list.keys()
#header += list(indicator_list.keys())
results.append(header)

print(results)

## Request worldbank!

In [178]:
import pandas
import requests

In [179]:
indicator_list={
'IMF WB\nCustoms and other import duties (% of tax revenue) 2016':'GC.TAX.IMPT.ZS',
'WB IMF Non-performing loans to total gross loans ratio (%)':'FB.AST.NPER.ZS',
'UIC Rail freight (mn ton x km travelled)':'IS.RRS.GOOD.MT.K6',
'WB Household final consumption expenditure, PPP bn USD':'NE.CON.PRVT.PP.CD',
'WB UNESCO R&D Expenditure % of GDP':'GB.XPD.RSDV.GD.ZS',
'WB UNESCO Researchers p. mn inhabitants':'SP.POP.SCIE.RD.P6',
'IFDI Net Inflows (% of GDP)':'BX.KLT.DINV.WD.GD.ZS',
'WHO Current Health Expenditure p.c., PPP USD':'SH.XPD.CHEX.PP.CD',
'WB Research Expenditure % of GDP':'GB.XPD.RSDV.GD.ZS',
'WIPO Trademark applications':'IP.TMK.RSCT',
'WIPO Patent Applications':'IP.PAT.RESD'
}

In [180]:
def get_indicator(country_code, indicator_code):
    url = 'http://api.worldbank.org/v2/country/'
    url += country_code
    url += '/indicator/'
    url += indicator_code
    url += '?format=json'
    r = requests.get(url)
    try:
        data = r.json()
        if data[1] is None:
            return None
        else:
            for i in data[1]:
                if i['value'] is None:
                    continue
                else:
                    return i['value']
    except KeyError:
        return None
    except IndexError:
        return None

In [183]:
filename = 'C:\\Users\\Cyril\\Documents\\GitHub\\US02_Python\\Data-sheet-STUDENTS.xlsx'
df_v2 = pandas.read_excel(filename, header = 2, usecols = 'B:AR')

In [186]:
list = ['Country (according to CN Gov):']
for key in indicator_list:
    indicator_code = indicator_list[key]
    list.append(key) #Créer la liste des entêtes
    a = 0 #La variable permet d'incrémenter les lignes du tableau
    print(key)
    for tr in trs: #Pour chaque la ligne dans le tableau
        values = [td.string for td in tr.find_all('td')]
        country_name = str(values[0])
        df.at[a, 'Country (according to CN Gov):'] = country_name
        country_code = str(values[3])
        value = get_indicator(country_code, indicator_code)
        df_v2.at[a, key] = value
        if value is not None:
            df_v2.at[a, key] = value
        a = a + 1

IMF WB
Customs and other import duties (% of tax revenue) 2016
WB IMF Non-performing loans to total gross loans ratio (%)
UIC Rail freight (mn ton x km travelled)
WB Household final consumption expenditure, PPP bn USD
WB UNESCO R&D Expenditure % of GDP
WB UNESCO Researchers p. mn inhabitants
IFDI Net Inflows (% of GDP)
WHO Current Health Expenditure p.c., PPP USD
WB Research Expenditure % of GDP
WIPO Trademark applications
WIPO Patent Applications


In [199]:
df_v2 = df_v2[list]

,Country (according to CN Gov):,IMF WB Customs and other import duties (% of tax revenue) 2016,WB IMF Non-performing loans to total gross loans ratio (%),UIC Rail freight (mn ton x km travelled),"WB Household final consumption expenditure, PPP bn USD",WB UNESCO R&D Expenditure % of GDP,WB UNESCO Researchers p. mn inhabitants,IFDI Net Inflows (% of GDP),"WHO Current Health Expenditure p.c., PPP USD",WB Research Expenditure % of GDP,WIPO Trademark applications,WIPO Patent Applications
0,Afghanistan,32.551072,12.201830,NaN,5.271768e+10,NaN,NaN,0.273179,183.927858,NaN,NaN,NaN
1,Albania,2.216459,13.232533,46.000,2.400144e+10,0.15412,156.101105,7.839316,773.678292,0.15412,918.0,21.0
2,Armenia,6.275395,5.432318,640.400,1.989999e+10,NaN,NaN,2.164932,883.220652,NaN,2356.0,125.0
3,Azerbaijan,4.550383,NaN,6210.500,9.360764e+10,NaN,NaN,7.037159,1191.289459,NaN,2817.0,144.0
4,Bahrain,87.914439,NaN,NaN,2.366161e+10,0.10116,368.902344,1.469627,2453.155914,0.10116,371.0,6.0
5,Bangladesh,28.923008,8.397908,710.000,3.756562e+11,NaN,NaN,0.861500,88.035356,NaN,8580.0,77.0
6,Belarus,8.200619,12.852194,41107.000,8.166474e+10,NaN,NaN,2.343707,1084.619757,NaN,4652.0,455.0
7,Bhutan,3.052294,8.417308,NaN,3.387944e+09,NaN,NaN,-0.654814,287.128628,NaN,16.0,3.0
8,Bosnia and Herzegovina,0.066132,10.046275,1139.220,3.124688e+10,0.21598,404.399902,2.562925,1101.792914,0.21598,797.0,60.0
9,Brunei,NaN,3.538399,NaN,4.203733e+09,0.03701,283.431091,3.858213,2083.430502,0.03701,85.0,26.0


In [195]:
#Enregistre un fichier Data_v2 avec les données du tableau df
with open ('C:\\Users\\Cyril\\Documents\\GitHub\\US02_Python\\Data_v2.pickle','wb+') as fp:
    pickle.dump(df_v2,fp)

In [196]:
#Charge le fichier Data_v2
with open ('C:\\Users\\Cyril\\Documents\\GitHub\\US02_Python\\Data_v2.pickle','rb') as fp:
    df_v2 = pickle.load(fp)

In [197]:
#Ecrit un fichier excel
df_v2.to_excel('final-result_v2.xls')